# Les unités de mesures utilisées sont distances : metres , temps : minutes , vitesse : metres / minutes dans un souci d'uniformisation

# Étape 1: Création des nœuds pour les stations

In [ ]:
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/pauldechorgnat/cool-datasets/master/ratp/stations.csv' AS row
CREATE (s:Station {
    name: row.nom_clean, 
    line: toInteger(row.ligne), 
    lat: toFloat(row.x), 
    lon: toFloat(row.y), 
    nom_gare: row.nom_gare, 
    traffic: toInteger(row.Trafic), 
    city: row.Ville
});

# Étape 2: Création des relations entre stations de la même ligne

In [ ]:
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/pauldechorgnat/cool-datasets/master/ratp/liaisons.csv' AS row
MATCH (start:Station {name: row.start}), (stop:Station {name: row.stop})
WHERE start.line = toInteger(row.ligne) AND stop.line = toInteger(row.ligne)
MERGE (start)-[:CONNECTS_TO {mode: 'train', speed: 2000/3}]->(stop);

# Étape 3: Création des relations de correspondance entre les lignes d'une même station

In [ ]:
MATCH (s1:Station), (s2:Station)
WHERE s1.name = s2.name AND s1.line <> s2.line
MERGE (s1)-[:CORRESPONDENCE {time: 4}]->(s2);

# Étape 4: Création des relations pour les déplacements à pied

In [ ]:
MATCH (s1:Station), (s2:Station)
WHERE s1 <> s2 AND SQRT((s1.lat - s2.lat)^2 + (s1.lon - s2.lon)^2) < 1000
MERGE (s1)-[:WALK_TO {speed: 200/3}]->(s2);